## 10. Introduction to Artificial Neural Networks with Keras

### ### Building Complex Models using the Functional API

Let's build a wide & deep network to tackle the **housing prices** problem

In [1]:
from keras.datasets import boston_housing

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, train_test_split

from tensorflow import keras as tf_keras

Using TensorFlow backend.


In [2]:
# Ingestion
###########
(train_data, y_train), (test_data, y_test) = boston_housing.load_data()

# Preprocessing
###############
sc = StandardScaler()
x_train = sc.fit_transform(train_data)
x_test = sc.transform(test_data)

x_train__train, x_train__val, y_train__train, y_train__val = train_test_split(x_train, y_train, test_size=0.15,
                                                                             random_state=0)
NUM_FEATURES = x_train.shape[1:]

An example of a non-sequential neural network is the <b>Wide & Deep</b> neural network. This architecture connects all or part of the inputs directly to the output layer. With this architecture, it is possible to learn both deep patterns  (using the deep path) and simple rules (using the short path). 

<img src="img3.png" width="900"/>

In contrast, a regular network forces all the data to flow through all layers, and simple patterns might end up being distorted.

In [3]:
# Instantiate Model
###################
input_layer = tf_keras.layers.Input(shape=NUM_FEATURES)
hidden_layer1 = tf_keras.layers.Dense(30, activation='relu')(input_layer)
hidden_layer2 = tf_keras.layers.Dense(30, activation='relu')(hidden_layer1)
concat_layer = tf_keras.layers.Concatenate()([hidden_layer1, hidden_layer2])
output_layer = tf_keras.layers.Dense(1)(concat_layer)
model0 = tf_keras.models.Model(inputs=[input_layer], outputs=output_layer)

In [4]:
model0.compile(optimizer='sgd', loss='mean_squared_error', metrics=['mae'])
history0 = model0.fit(x_train, y_train,  epochs = 10, verbose=0)

In [5]:
# Instantiate Model
###################
input_layera = tf_keras.layers.Input(shape=(10,))
input_layerb = tf_keras.layers.Input(shape=(7,))
hidden_layer1 = tf_keras.layers.Dense(30, activation='relu')(input_layerb)
hidden_layer2 = tf_keras.layers.Dense(30, activation='relu')(hidden_layer1)
concat_layer = tf_keras.layers.Concatenate()([input_layera, hidden_layer2])
output_layer = tf_keras.layers.Dense(1)(concat_layer)
model1 = tf_keras.models.Model(inputs=[input_layera, input_layerb], outputs=output_layer)

In [6]:
# Prepare data for feeding to NN 
####
x_train__trainA = x_train__train[:,:10]
x_train__trainB = x_train__train[:,[1,5,6,7,8,11,12]]
x_train__val_A = x_train__val[:,:10]
x_train__val_B = x_train__val[:,[1,5,6,7,8,11,12]]

In [7]:
model1.compile(optimizer='sgd', loss='mean_squared_error', metrics=['mae'])

In [8]:
model1.fit((x_train__trainA, x_train__trainB), y_train__train, epochs=20,
           validation_data=((x_train__val_A, x_train__val_B), y_train__val), verbose=0)

In [9]:
x_testA = x_test[:,:10]
x_testB = x_test[:,[1,5,6,7,8,11,12]]

In [10]:
model1.evaluate((x_testA, x_testB), y_test)

4/4 [==============================] - 0s 1ms/step - loss: 41.8889 - mae: 5.2461


[41.88889694213867, 5.246099948883057]

In [11]:
model1.predict((x_testA[:2], x_testB[:2]))

array([[ 6.039479],
       [14.863385]], dtype=float32)

### Using the Subclassing API

In [12]:
class WideAndDeepModel(tf_keras.models.Model):
    def __init__(self, units=30, activation='relu', **kwargs):
        super().__init__(**kwargs)
        self.hidden_layer1 = tf_keras.layers.Dense(units, activation=activation)
        self.hidden_layer2 = tf_keras.layers.Dense(units, activation=activation)
        self.output_layer = tf_keras.layers.Dense(1)
    
    def call(self, inputs):
        inputa, inputb = inputs
        hidden1 = self.hidden_layer1(inputb)
        hidden2 = self.hidden_layer2(hidden1)
        conct = tf_keras.layers.Concatenate()([inputa, hidden2])
        ouptt = self.output_layer(conct)
        return ouptt
        

In [13]:
model3 = WideAndDeepModel(30, 'relu')
model3.compile(optimizer='sgd', loss='mean_squared_error', metrics=['mae'])
model3.fit((x_train__trainA, x_train__trainB), y_train__train, epochs=20,
           validation_data=((x_train__val_A, x_train__val_B), y_train__val), verbose=0)

In [14]:
model3.evaluate((x_testA, x_testB), y_test)

4/4 [==============================] - 0s 1ms/step - loss: 31.1054 - mae: 3.4079


[31.105440139770508, 3.407899856567383]

### Saving & Restoring a Model

In [15]:
model3.predict((x_testA[:2], x_testB[:2]))

array([[11.237417],
       [19.023613]], dtype=float32)

In [16]:
model1.save('model3.h5')

In [17]:
model1ld = tf_keras.models.load_model('model3.h5')
model1ld.predict((x_testA[10:15], x_testB[10:15]))

array([[16.940674],
       [15.326251],
       [14.198267],
       [34.23001 ],
       [10.630935]], dtype=float32)

Additional Readings:

- https://ai.googleblog.com/2016/06/wide-deep-learning-better-together-with.html